In [1]:
import pandas as pd
from mrich import print
from pathlib import Path
from datetime import datetime

In [15]:
data = {}
for file in Path("data/InVivo2/spect").glob("*/*.csv"):
    
    df = pd.read_csv(file, skiprows=1)
    df = df[df["Name"] == "Heart"]
    
    date, rat, timepoint, _ = file.name.split("_")
    
    rat = int(rat.removeprefix("R"))
    date = datetime.strptime(date, "%Y%m%d").date()
    timepoint = timepoint.replace("wk", " wk")
    
    base_d = {
        "Date":date,
        "Timepoint":timepoint,
        "Rat":rat,
    }
    
    for i,row in df.iterrows():
        
        scan = "30m" if "30m" in row["Series"] else "4h"
        
        d = data.setdefault((scan, timepoint, rat), dict(base_d))
        
        d["Scan"] = scan
        
        key = f"{row['Image name']}"
        
        for col in df.columns[3:]:
            d[f"{key} [{col}]"] = row[col]
    
        # data.append(d)
    
print(len(data))
    
df = pd.DataFrame(data.values())

df["Treatment"] = df["Rat"].apply(lambda x: "Control" if x%4<2 else "IR")

indices = ["Scan", "Timepoint", "Treatment", "Rat"]
scans = ["30m", "4h"]
timepoints = ["Baseline", "4 wk", "8 wk", "10 wk", "14 wk", "18 wk", "22 wk", "25 wk"]

df["Timepoint"] = pd.Categorical(df["Timepoint"], categories=timepoints, ordered=True)
df["Scan"] = pd.Categorical(df["Scan"], categories=scans, ordered=True)
    
df = df.sort_values(by=indices)
df = df.set_index(indices)

display(df)

60

Date  \
Scan Timepoint Treatment Rat               
30m  Baseline  Control   64   2025-05-03   
                         69   2025-05-03   
                         72   2025-05-03   
                         73   2025-05-03   
                         77   2025-05-03   
               IR        70   2025-05-03   
                         71   2025-05-03   
                         74   2025-05-03   
                         75   2025-05-03   
                         79   2025-05-03   
     8 wk      Control   64   2025-05-03   
                         69   2025-05-03   
                         72   2025-05-03   
                         73   2025-05-03   
                         77   2025-05-03   
               IR        70   2025-05-03   
                         71   2025-05-03   
                         74   2025-05-03   
                         75   2025-05-03   
                         79   2025-05-03   
     25 wk     Control   64   2025-05-03   
                         69   2025-05-03   
                         72   2025-05-03   
                         73   2025-05-03   
                         77   2025-05-03   
               IR        70   2025-05-03   
                         71   2025-05-03   
                         74   2025-05-03   
                         75   2025-05-03   
                         79   2025-05-03   
4h   Baseline  Control   64   2025-05-03   
                         69   2025-05-03   
                         72   2025-05-03   
                         73   2025-05-03   
                         77   2025-05-03   
               IR        70   2025-05-03   
                         71   2025-05-03   
                         74   2025-05-03   
                         75   2025-05-03   
                         79   2025-05-03   
     8 wk      Control   64   2025-05-03   
                         69   2025-05-03   
                         72   2025-05-03   
                         73   2025-05-03   
                         77   2025-05-03   
               IR        70   2025-05-03   
                         71   2025-05-03   
                         74   2025-05-03   
                         75   2025-05-03   
                         79   2025-05-03   
     25 wk     Control   64   2025-05-03   
                         69   2025-05-03   
                         72   2025-05-03   
                         73   2025-05-03   
                         77   2025-05-03   
               IR        70   2025-05-03   
                         71   2025-05-03   
                         74   2025-05-03   
                         75   2025-05-03   
                         79   2025-05-03   

                                                              CT( HU ) [Series]  \
Scan Timepoint Treatment Rat                                                      
30m  Baseline  Control   64   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         69   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         72   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         73   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         77   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
               IR        70   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         71   BR_CTRecon,30m CT Scan (2),FBP,V250,Sl250,F:Co...   
                         74   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         75   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         79   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
     8 wk      Control   64   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         69   BR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         72   NR_CTRecon,30m CT Scan,FBP,V250,Sl250,F:Cos,C1...   
                         73   BR_CTRecon,30m CT Scan (2),FBP,V

In [16]:
df.to_excel("InVivo2_spect_processed.xlsx")